In [1]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

/Users/pg47711/Downloads/langgraph-bootcamp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from langchain_community.llms import Ollama

In [5]:
llm = Ollama(model="gemma3:4b")

/var/folders/rt/8nwykg3905sf1dpt84_r026c0000gp/T/ipykernel_77344/1559220570.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b")


In [9]:

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [10]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='43c6f64f-80d7-466d-8b43-e48cb2b37842'),
 HumanMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={}, id='a3752910-b46a-460c-8d96-6867c492d821')]

In [11]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Neil Armstrong was the first person to walk on the Moon! He took that historic step on July 20, 1969, as part of the Apollo 11 mission. 

Buzz Aldrin followed him shortly after. 😊 

Do you want to know anything more about the Apollo 11 mission or the first moonwalk?
Bot: Okay, great! Let’s delve into the Apollo 11 mission and the first moonwalk. 

Here’s some more information:

*   **The Mission’s Goal:** The primary goal of Apollo 11 was to land humans on the Moon and return them safely to Earth. It was a massive undertaking driven by the Cold War space race with the Soviet Union.

*   **The Crew:** Besides Neil Armstrong and Buzz Aldrin, the crew consisted of Michael Collins, who piloted the command module *Columbia* in orbit around the Moon, waiting for the lunar module to return.

*   **The Descent and First Step:** Armstrong was the first to descend to the lunar surface in the lunar module *Eagle*. The famous first step was a carefully planned moment, and Armstrong’s words, “